## ⚠️ WARNING

This notebook isn't directly used for the API and is only intended for the development process of the project. 

Please remove the following to ensure a clean production build

```bash
uv remove ollama
```

## Ollama Assistance

This section is specialized in utilizing local llms such as Ollama's `llama3.2` model to create documentations. This works in tandem to [data_preparations.ipynb](data_preparations.ipynb) for text embedding the docstrings of a function inside the `./submission` module.

The purpose of creating this localized assistance is to generate texts without dealing with internet or server issues from other GPT models and capping out token usages.


### Installing ollama

```bash
curl -fsSL https://ollama.com/install.sh | sh
```


### Installing a model

```bash
ollama run llama3.2
```

### Ollama Python Library

```bash
uv add ollama
```

## Initialization and System Prompt

In [ ]:
system_prompt = """
    You create docstrings for functions provided by the user. These are the rules for your output
    - Keep the docstrings short but very specific and clear as to what the function does. 
    - Ensure that this content is in Google Python Style.
    - Ensure that it us a valid docstring for the ast Python library.
    - You only have to provide the docstring, there's no need to write the code only the docstring, use this format below as a guide for your answer.
    - Do a double check on your output before providing an answer.

    \"""
    docstring
    
    Args:
        param1 (str): A description of param1
        param2 (str): A description of param2
    
    Returns:
        int: A description of the output
    
    Raises:
        Type of error: A description of the error
    
    Example:
        >>> example("hello", "world")
        hello world
    \"""
"""

In [ ]:
import ollama

def ask_something(prompt: str):
    response = ollama.chat(
        model = "llama3.2",
        messages=[
            { "role": "user", "content": prompt }
        ]
    )
    return str(response["message"]["content"])

def create_docstring(prompt: str):
    response = ollama.chat(
       model = "llama3.2",
        messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": prompt }
        ]
    )
    return str(response["message"]["content"])


## Docstring exports

In [ ]:
print(create_docstring("""
def yt_transcribe(url: str) -> str:
    def clean_transcripts(df_export: pd.DataFrame) -> list[str]:
        export = df_export.to_string(index=False)
        export = export.split("\n")
        for idx, text in enumerate(export):
            export[idx] = text.strip()
        return export
    def download_audio(url: str, filename: str) -> None:
        ydl_opts = {
            "format": "ba[abr<50]/worstaudio",
            "postprocessors": [{
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "32"
            }],
            "outtmpl": f"./data/{video_code}.%(ext)s",
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    video_code = url.split("=")[1]
    if not os.path.isfile(f"./data/{video_code}.mp3"):
        download_audio(url, video_code)
    audio = open(f"./data/{video_code}.mp3", "rb")
    transcription = client.audio.transcriptions.create(
        file=audio,
        model="whisper-1",
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )
    words = json.loads(transcription.to_json())["words"]
    df = pd.DataFrame(words)
    clip = df[(df["start"] >= 173.4) & (df["end"] <= 229.7)]["word"]
    count = []
    for i in clean_transcripts(clip):
        if ord(i[0]) <= 90 and not (i == "Miranda" or i == "Edmund"):
            count.append(".")
        count.append(i)
    return str(len(count))
"""))

## Ask something

In [ ]:
print(ask_something("""
#!/bin/bash

# Function to increment each digit in a given string
increment_digits() {
	echo "$1" | tr '0123456789' '1234567890'
}

# Loop through all files in the directory
for FILE in *; do
	# Check if the file has numbers
	if [[ $FILE =~ [0-9] ]]; then
    	# Replace numbers with incremented numbers
    	NEW_FILE=$(increment_digits "$FILE")
   	 
    	# Check if the new filename already exists to avoid overwriting
    	if [[ -e $NEW_FILE ]]; then
        	echo "Cannot rename $FILE to $NEW_FILE because $NEW_FILE already exists."
    	else
        	mv "$FILE" "$NEW_FILE"
    	fi
	fi
done

give me the python code equivalent to this bash script
"""))